Definição: o processo de revisar e limpar dados para obtermos insights (como estatística descritiva e correlação) e gerar hipóteses.

## Conhecer o dataset

In [2]:
import pandas as pd
df = pd.DataFrame()

In [ ]:
df.info() # non-nulls, object types
df['column'].value_counts() # value counts
df['column'].describe() # statistics from this variable
df.describe() # statistics for each numerical column

In [ ]:
# visualização: seaborn e matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

sns.histplot(data=df, x='column', binwidth=.1)
plt.show()

### Data validation

- Verificar se õs dados estão dentro dos intervalos esperados.

In [ ]:
df.info() # função que dá os datatypes e contagem de valores não nulos de cada coluna
df.dtypes # atribuito que dá apenas os datatypes das colunas

# editing datatypes:
df['column'] = df['column'].astype(int)

In [ ]:
# validando dados categóricos
df['column'].unique()
df['column'].isin(['value_1','value_2']) # series de bool; pode ser usado pra filtrar
~df['column'].isin(['value_1','value_2']) # '~' inverte True com False

In [ ]:
# validando dados numéricos:
df.select_dtypes("number") # filtra o df apenas para colunas numéricas
df['column'].min()
df['column'].max()

# boxplot:
sns.boxplot(data=df, x='column')
plt.show()

# multiplos boxplots (uma variável agrupada por uma outra variável categórica)
sns.boxplot(data=df, x='numeric_column', y='categorical_column')
plt.show()


### Data Sumarization

In [ ]:
df.groupby("categorical_column").mean() # pega a média de todas as colunas numéricas
# outras funções agregadoras: .sum(), .count(), .min(), .max(), .var(), .std()   e .agg()

# .agg() permite selecionar funções diferentes:
df.groupby("categorical_column").agg(["mean", "std"]) # lista: pega a média e o desvio pra cada variável numérica
df.groupby("categorical_column").agg({"column2": "count", "column3": "sum"}) # dict: chave determina a coluna e o valor determina a função
df.groupby("categorical_column").agg(mean_column2=("column2", "mean"),      # tuples: nome é o nome da coluna a ser criada, 
                                    std_column2=("column2", "std"),                   # primeiro argumento é a variável a ser tirada a métrica 
                                    median_column3=("column3", "median"))            # terceiro é a métrica a ser tirada
                                                                                      # lembrando que as linhas serão as categorias agrupadas

In [ ]:
# barplot: semelhante ao groupby, mostra média e barra de erro mostra std
sns.barplot(data=df, x='categorical_column', y='numerical_column')
plt.show()

## Tratamento

### Ausência de dados

- Remover os registros se eles representarem  ausencia maior igual a 5% em uma variável;
- Imputação: substituir por alguma estatística;
- Imputar por subgrupo: pegar a mediana de cada categoria de uma variável escolhida;


In [ ]:
print(df.isna().sum()) # soma os booleanos True de dados faltantes por coluna

In [ ]:
# dropar colunas:
limite = len(df) * 0.05 # pega somente o len vertical
cols_to_drop = df.columns[df.isna().sum() <= limite]
df = df.dropna(subset=cols_to_drop)

In [ ]:
# imputing:
cols_with_nas = df.columns[df.isna().sum() < 0]

for col in cols_with_nas:
    df[col].fillna(df[col].median()[0]) # pra que esse `[0]`?

In [ ]:
# imputing by subgroup:
median_dict = df.groupby("categorical_column")['variable'].median().to_dict()
    # dicionário com o nome da categoria: mediana de variable para cada categoria

df['variable'] = df['variable'].fillna(df['categorical_column'].map(median_dict))